In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../dataset/Auto.csv')
df['horsepower'] =df['horsepower'].replace('?',0)
df['horsepower'] =df['horsepower'].astype(int)
y = df['displacement']

del df['name']

In [4]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1


In [5]:
#split it into train and test 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train , y_test = train_test_split(df, y,test_size = 0.33 , random_state = 1)

X_train = pd.DataFrame(X_train, columns = df.columns)
X_test = pd.DataFrame(X_test, columns = df.columns)


In [28]:
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score

In [37]:
#Create dataset of lightgbm

# lgb_train = lgb.Dataset(X_train, y_train)
# lgb_eval = lgb.Dataset(X_test, y_test)

#get number of feature and number of training rows
num_train, num_feature = X_train.shape

#Get parameters 
params = {
#     'eta': 0.02,
                  'max_depth': 4,
#                   'subsample': 0.9,
#                   'colsample_bytree': 0.9,
#                   'objective': 'binary:logistic',
#                   'eval_metric': 'auc',
                  'seed': 99,
                  'silent': True}

#------------------------------------------------------------
folds = KFold(n_splits=2, shuffle=True, random_state=123)

oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(X_test.shape[0])
feature_importance_df = pd.DataFrame()
feature_name = [col for col in X_train.columns]
print(feature_name)
print('Starting training...')
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train)):
    print('fold {} '.format(n_fold))
    trn_x, trn_y = X_train[feature_name].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = X_train[feature_name].iloc[val_idx], y.iloc[val_idx]
    xgb_train = xgb.DMatrix(trn_x, trn_y)
    xgb_eval = xgb.DMatrix(val_x, val_y)
    xgb_test = xgb.DMatrix(X_test[feature_name])
    
    watchlist = [(xgb_train, 'train'), (xgb_eval, 'eval')]
    
    gbm = xgb.train(params,xgb_train,feval=gini_xgb,verbose_eval = -1)
    
    oof_preds[val_idx] = gbm.predict(xgb.DMatrix(val_x)) #get oof prediction
    
    #predict on test set, take average
    sub_preds += gbm.predict(xgb_test) / folds.n_splits 
    
    #save the feature important 
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = feature_name
#     fold_importance_df["importance"] = np.log1p(gbm.feature_importances_)
#     df = pd.DataFrame(gbm.get_fscore().items(), columns=['feature','importance']).sort_values('importance', ascending=False)

#     fold_importance_df["fold"] = n_fold + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']
Starting training...
fold 0 
fold 1 
